# Day 6: Sequence Models in Deep Learning

### Exercise 6.1 
Convince yourself a RNN is just an MLP with inputs and outputs at various layers. Run the NumpyRNN code. Set break-points and compare with what you learned about back-propagation in the previous day.

Start by loading data Part-of-speech data and configure it for the exercises

In [1]:
# Add tools
# NOTE: This should only be needed if you do not store the notebook on the lxmls root
import sys
sys.path.append('../../')
from pdb import set_trace

In [2]:
# Location of Part-of-Speech WSJ Data
WSJ_TRAIN = "../../data/train-02-21.conll"
WSJ_TEST = "../../data/test-23.conll"
WSJ_DEV = "../../data/dev-22.conll"

In [3]:
# Load Part-of-Speech data 
import lxmls.readers.pos_corpus as pcc
corpus = pcc.PostagCorpus()
train_seq = corpus.read_sequence_list_conll(WSJ_TRAIN, max_sent_len=15, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll(WSJ_TEST, max_sent_len=15, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll(WSJ_DEV, max_sent_len=15, max_nr_sent=1000) 
# Redo indices so that they are consecutive. Also cast all data to numpy arrays
# of int32 for compatibility with GPUs and theano and add reverse index
train_seq, test_seq, dev_seq = pcc.compacify(train_seq, test_seq, dev_seq, theano=True)
# Get number of words and tags in the corpus
nr_words = len(train_seq.x_dict)
nr_tags = len(train_seq.y_dict)

In [9]:
# We start with random embeddings
import numpy as np
E = np.random.randn(50, nr_words)

Model configuration

In [10]:
import lxmls.deep_learning.rnn as rnns
reload(rnns)

<module 'lxmls.deep_learning.rnn' from '../../lxmls/deep_learning/rnn.pyc'>

In [11]:
# RNN configuration
SEED = 1234       # Random seed to initialize weigths
hidden_size = 20  # size of hidden layer

In [12]:
np_rnn = rnns.NumpyRNN(E, hidden_size, nr_tags, seed=SEED)

In [13]:
x0 = train_seq[0].x
y0 = train_seq[0].y
loos, p_y, y_rnn, h, z1, x = np_rnn.forward(x0, all_outputs=True, 
                                            outputs=y0)

In [14]:
# Compute gradients
numpy_rnn_gradients = np_rnn.grads(x0, y0)

### Exercise 6.2
Understand the basics of scan with this examples. Scan allows you to build computation graphs with a variable number of nodes. It acts as a python "for" loop but it is symbolic. The following example should help you understand the basic scan functionality. It generates a sequence for a given length. Run it and modify it. Try to arrive at an error.

In [15]:
import numpy as np
import theano
import theano.tensor as T
theano.config.optimizer='None'

In [16]:
def square(x): 
    return x**2 

# Python
def np_square_n_steps(nr_steps):
    out = []
    for n in np.arange(nr_steps):
        out.append(square(n))
    return np.array(out)
    
# Theano
nr_steps = T.lscalar('nr_steps')
h, _ = theano.scan(fn=square, sequences=T.arange(nr_steps))
th_square_n_steps = theano.function([nr_steps], h)

In [17]:
print np_square_n_steps(10)
print th_square_n_steps(10)

[ 0  1  4  9 16 25 36 49 64 81]
[ 0  1  4  9 16 25 36 49 64 81]


The following example should help you understand about matrix multiplications and passing values from one iteration to the other. It at each step it we multiply the output of the previous step by a matrix A. We start with an initial vector s0. The matrix and vector are random but normalized to result on a Markov chain.  

In [18]:
# Configuration
nr_states = 3
nr_steps = 5

# Transition matrix
A = np.abs(np.random.randn(nr_states, nr_states))
A = A/A.sum(0, keepdims=True)
# Initial state
s0 = np.zeros(nr_states)
s0[0] = 1

In [19]:
# Numpy version
def np_markov_step(s_tm1): 
    s_t = np.dot(s_tm1, A.T)
    return s_t 

def np_markov_chain(nr_steps, A, s0):
    # Pre-allocate space
    s = np.zeros((nr_steps+1, nr_states))
    s[0, :] = s0
    for t in np.arange(nr_steps):
        s[t+1, :] = np_markov_step(s[t, :])
    return  s           

In [20]:
np_markov_chain(nr_steps, A, s0)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.53422672,  0.21598476,  0.24978852],
       [ 0.51121322,  0.20438911,  0.28439767],
       [ 0.51045292,  0.20224872,  0.28729836],
       [ 0.51045754,  0.20205431,  0.28748815],
       [ 0.51046102,  0.20204089,  0.28749809]])

In [21]:
# Theano version
# Store variables as shared variables
th_A = theano.shared(A, name='A', borrow=True)
th_s0 = theano.shared(s0, name='s0', borrow=True)
# Symbolic variable for the number of steps
th_nr_steps = T.lscalar('nr_steps')

def th_markov_step(s_tm1): 
    s_t = T.dot(s_tm1, th_A.T)
    # Remember to name variables
    s_t.name = 's_t'
    return s_t 

s, _ = theano.scan(th_markov_step, 
                   outputs_info=[dict(initial=th_s0)], 
                   n_steps=th_nr_steps)
th_markov_chain = theano.function([th_nr_steps], T.concatenate((th_s0[None, :], s), 0))

In [22]:
th_markov_chain(nr_steps)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.53422672,  0.21598476,  0.24978852],
       [ 0.51121322,  0.20438911,  0.28439767],
       [ 0.51045292,  0.20224872,  0.28729836],
       [ 0.51045754,  0.20205431,  0.28748815],
       [ 0.51046102,  0.20204089,  0.28749809]])

### Exercise 6.3
Complete the theano code for a RNN inside lxmls/deep learning/rnn.py. Use exercise 6.1 for a numpy example and 6.2 to learn how to handle scan. Keep in mind that you only need to implement the forward pass!. Theano will handle backpropagation ofr us. 

In [23]:
rnn = rnns.RNN(E, hidden_size, nr_tags, seed=SEED)

In [24]:
# Compile theano function
x = T.ivector('x')
th_forward = theano.function([x], rnn._forward(x).T)

When working with theano, it is more difficult to localize the source of errors. It is therefore extremely important to work step by step and test the code frequently. To debug we suggest to implement and compile the forward pass first. You can use this code for testing. If it raises no error you are good to go.

In [25]:
assert np.allclose(th_forward(x0), np_rnn.forward(x0)), \
    "Numpy and Theano forward pass differ!"

Once you are confident the forward pass is working you can test the gradients

In [26]:
# Compile function returning the list of gradients
x = T.ivector('x')     # Input words
y = T.ivector('y')     # gold tags 
p_y = rnn._forward(x)
cost = -T.mean(T.log(p_y)[T.arange(y.shape[0]), y])
grads_fun = theano.function([x, y], [T.grad(cost, par) for par in rnn.param])

In [27]:
# Compare numpy and theano gradients
theano_rnn_gradients = grads_fun(x0, y0)
for n in range(len(theano_rnn_gradients)): 
    assert np.allclose(numpy_rnn_gradients[n], theano_rnn_gradients[n]), \
    "Numpy and Theano gradients differ in step n"

Finally, its time to test our network!. For this, lets first compile a function that does predictions

In [28]:
rnn_prediction = theano.function([x], T.argmax(p_y, 1))
# Lets test the predictions
sample_seq = train_seq[0]
words = [train_seq.word_dict[wrd] for wrd in sample_seq.x]
tags = [train_seq.tag_dict[pred] for pred in rnn_prediction(sample_seq.x)]
print ["/".join([word, tag]) for word , tag in zip(words, tags)]

[u'Ms./det', u'Haag/det', u'plays/det', u'Elianti/det', u'./det']


Now lets define the optimization parameters and compile a batch update function

In [29]:
lrate = 0.5
n_iter = 5

In [30]:
# Get list of SGD batch update rule for each parameter
updates = [(par, par - lrate*T.grad(cost, par)) for par in rnn.param]
# compile
rnn_batch_update = theano.function([x, y], cost, updates=updates)

Finally it is time to run SGD. You can use the following code for this purpose

In [ ]:
nr_words = sum([len(seq.x) for seq in train_seq])
for i in range(n_iter):
    
    # Training
    cost = 0
    errors = 0
    for n, seq in enumerate(train_seq):
        cost += rnn_batch_update(seq.x, seq.y)
        errors += sum(rnn_prediction(seq.x) != seq.y)
    acc_train = 100*(1-errors*1./nr_words) 
    print "Epoch %d: Train cost %2.2f Acc %2.2f %%" % (i+1, cost, acc_train), 
    
    # Evaluation    
    errors = 0
    for n, seq in enumerate(dev_seq):
        errors += sum(rnn_prediction(seq.x) != seq.y)  
    acc_dev = 100*(1-errors*1./nr_words) 
    print " Devel Acc %2.2f %%" % acc_dev
    sys.stdout.flush()

Epoch 1: Train cost 2805.93 Acc 29.99 %  Devel Acc 75.13 %
Epoch 2: Train cost 2809.57 Acc 30.03 %  Devel Acc 75.13 %


In [32]:
rnn = rnns.reset_model(rnn, seed=SEED)

In [ ]:
# Reset model to remove the effect of training§
rnn = rnns.reset_model(rnn, seed=SEED)

# Embeddings Path
EMBEDDINGS = "../../data/senna_50"
import lxmls.deep_learning.embeddings as emb
import os
reload(emb)
if not os.path.isfile(EMBEDDINGS):
    emb.download_embeddings('senna_50', EMBEDDINGS)
E = emb.extract_embeddings(EMBEDDINGS, train_seq.x_dict)  

In [ ]:
rnns.

### Exercise 6.4
Convince yourself that LSTMs are just more complex RNNs. Run them, play around with thehyper parameters and compare the RNN and LSTM classes.

In [ ]:
lstm = rnns.LSTM(E, hidden_size, nr_tags)
lstm_prediction = theano.function([x], 
                                  T.argmax(lstm._forward(x), 1))
lstm_cost = -T.mean(T.log(lstm._forward(x))[T.arange(y.shape[0]), y])

In [ ]:
# Get list of SGD batch update rule for each parameter
lstm_updates = [(par, par - lrate*T.grad(lstm_cost, par)) for par in lstm.param]
# compile
lstm_batch_update = theano.function([x, y], lstm_cost, 
                                    updates=lstm_updates)

In [ ]:
nr_words = sum([len(seq.x) for seq in train_seq])
for i in range(n_iter):
    
    # Training
    cost = 0
    errors = 0
    for n, seq in enumerate(train_seq):
        cost += lstm_batch_update(seq.x, seq.y)
        errors += sum(lstm_prediction(seq.x) != seq.y)
    acc_train = 100*(1-errors*1./nr_words) 
    print "Epoch %d: Train cost %2.2f Acc %2.2f %%" % (i+1, cost, acc_train), 
    
    # Evaluation    
    errors = 0
    for n, seq in enumerate(dev_seq):
        errors += sum(lstm_prediction(seq.x) != seq.y)  
    acc_dev = 100*(1-errors*1./nr_words) 
    print " Devel Acc %2.2f %%" % acc_dev
    sys.stdout.flush()